In [ ]:
!pip install bitsandbytes
!pip install peft
!pip install accelerate

In [1]:
file_path = "new_file.txt"
text_content = """
The Evolution of the Automobile Industry

Early Innovations:

The automobile industry has come a long way since its inception in the late 19th century. The first automobiles were powered by steam engines, but these were soon replaced by internal combustion engines, which proved to be more efficient and reliable. Karl Benz is often credited with creating the first true automobile in 1885, which was powered by a gasoline engine. This marked the beginning of a revolution in personal and commercial transportation.

Mass Production and the Model T:

A significant milestone in the automobile industry was the introduction of mass production techniques by Henry Ford in the early 20th century. Ford's Model T, launched in 1908, became the first affordable car for the average American. The assembly line production method not only reduced the cost of manufacturing but also significantly increased the speed of production. By 1927, Ford had produced over 15 million Model T cars, making it the best-selling car of its time.

Technological Advancements:

Throughout the 20th century, the automobile industry witnessed numerous technological advancements. The introduction of electric starters, automatic transmissions, and power steering greatly improved the convenience and safety of driving. The development of safety features such as seat belts, airbags, and anti-lock braking systems (ABS) also contributed to making cars safer for occupants.

Environmental Concerns and Fuel Efficiency:

In the latter half of the 20th century, growing concerns about environmental pollution and fuel efficiency led to significant changes in the industry. The oil crisis of the 1970s prompted automakers to focus on producing smaller, more fuel-efficient cars. The introduction of catalytic converters and stricter emissions standards helped reduce the environmental impact of automobiles.

The Rise of Electric and Hybrid Vehicles:

The 21st century has seen a renewed focus on reducing the environmental footprint of automobiles. Electric and hybrid vehicles have become increasingly popular as technology has improved and concerns about climate change have grown. Companies like Tesla have revolutionized the market with high-performance electric vehicles (EVs) that offer long driving ranges and advanced features. Traditional automakers are also investing heavily in EV technology, with many planning to phase out internal combustion engines entirely in the coming decades.

Autonomous Driving and the Future:

The future of the automobile industry is likely to be shaped by advancements in autonomous driving technology. Companies like Waymo, Uber, and Tesla are developing self-driving cars that promise to revolutionize transportation by reducing accidents, improving traffic flow, and providing new mobility options for those unable to drive. While fully autonomous vehicles are not yet widespread, ongoing developments suggest that they could become a common sight on our roads in the not-too-distant future.

Conclusion:

The automobile industry has undergone tremendous changes since the first cars were invented. From the early days of steam-powered vehicles to the latest advancements in electric and autonomous driving technology, the industry continues to evolve and adapt to meet the needs of consumers and address global challenges. As we move forward, the focus on sustainability and innovation will likely drive the next wave of transformations in this dynamic industry.

"""
# Open the file in write mode
with open(file_path, 'w') as file:
    # Write content to the file
    file.write(text_content)
     
print(f"File '{file_path}' created successfully.")

File 'new_file.txt' created successfully.


In [2]:
with open("/kaggle/working/new_file.txt","rb") as file:
    result = file.read()
print(result)

b"\nThe Evolution of the Automobile Industry\n\nEarly Innovations:\n\nThe automobile industry has come a long way since its inception in the late 19th century. The first automobiles were powered by steam engines, but these were soon replaced by internal combustion engines, which proved to be more efficient and reliable. Karl Benz is often credited with creating the first true automobile in 1885, which was powered by a gasoline engine. This marked the beginning of a revolution in personal and commercial transportation.\n\nMass Production and the Model T:\n\nA significant milestone in the automobile industry was the introduction of mass production techniques by Henry Ford in the early 20th century. Ford's Model T, launched in 1908, became the first affordable car for the average American. The assembly line production method not only reduced the cost of manufacturing but also significantly increased the speed of production. By 1927, Ford had produced over 15 million Model T cars, making i

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name='TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)#
#model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
 
#Quantisation
from transformers import BitsAndBytesConfig, Trainer, TrainingArguments
# Configure the model for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # For 8-bit quantization
    load_in_4bit=False  # Set to True if you want 4-bit quantization
)
# Configure the model for 8-bit quantization
#bnb_config = BitsAndBytesConfig.from_pretrained(model_name, load_in_8bit=True)
 
# Load the quantized model
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
##Adding trainable quantised peft adapter which will help in transfer learning through layer freezing
from peft import LoraConfig
 
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
model.add_adapter(peft_config)
# # Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
 
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
 
# Function to load dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset
 
# Function to create data collator
def create_data_collator(tokenizer):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # Masked Language Modeling (MLM) is false for autoregressive models like GPT-2
    )
    return data_collator
 
# Load dataset
train_dataset = load_dataset("/kaggle/working/new_file.txt", tokenizer)
 
# Create data collator
data_collator = create_data_collator(tokenizer)
 
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1000,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)
 
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)
 

2024-06-06 08:38:14.772668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 08:38:14.772763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 08:38:14.903302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.206900
1000,0.016700


TrainOutput(global_step=1000, training_loss=0.11177812671661377, metrics={'train_runtime': 506.9865, 'train_samples_per_second': 11.835, 'train_steps_per_second': 1.972, 'total_flos': 4808556675072000.0, 'train_loss': 0.11177812671661377, 'epoch': 1000.0})

In [5]:
model.save_pretrained("./fine-tuned-llama")
tokenizer.save_pretrained("./fine-tuned-llama")

/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./fine-tuned-llama/tokenizer_config.json',
 './fine-tuned-llama/special_tokens_map.json',
 './fine-tuned-llama/tokenizer.model',
 './fine-tuned-llama/added_tokens.json',
 './fine-tuned-llama/tokenizer.json')

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./fine-tuned-llama")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-llama")

# Check if the tokenizer is using padding side left or right
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Generate a response
prompt = "Environmental Concerns and Fuel Efficiency of automobile"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the model's response
output = model.generate(**inputs, max_length=1000)

# Decode the output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response,".........")


Environmental Concerns and Fuel Efficiency of automobile Industry

Fuel efficiency is one of the major environmental concerns of the modern era. The increasing pollution concerns have led to a need for more efficient cars and trucks. The automobile industry has responded by producing cars and trucks with higher fuel efficiency.

The early cars had a high fuel efficiency due to their design and construction. The internal combustion engines of those days were very efficient, and many cars of that time had over 20 mpg (million pounds per square inch) of efficiency. The war made use of more efficient engines, and cars with aluminum alloy bodies and lighter materials became more and more common.

Modern Technology and Fuel Type Efficiency

Fuel type efficiency is also a major concern of the modern era. Fuel cells and electric vehicles are becoming more and more common as technology improves. The latest vehicles often have high fuel efficiency due to their design and construction.

The adven

In [ ]:
prompt = "what is the education of MUHAMMED THAHSEER"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the model's response
output = model.generate(**inputs, max_length=100)

# Decode the output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response,".........")


In [18]:
prompt = "how automobile industry has undergone for a tremendous changes?"
inputs = tokenizer(prompt, return_tensors="pt")

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: val.to(device) for key, val in inputs.items()}
model.to(device)

# Generate tokens step-by-step
max_length = 500
generated = inputs["input_ids"]

for _ in range(max_length):
    outputs = model(generated)
    next_token_logits = outputs.logits[:, -1, :]
    next_token = torch.argmax(next_token_logits, dim=-1)
    generated = torch.cat((generated, next_token.unsqueeze(-1)), dim=-1)

    # Decode and print the latest token
    decoded_token = tokenizer.decode(next_token)
    print(decoded_token, end=" ", flush=True)

    # If the model generates the end of sentence token, stop early
    if next_token.item() == tokenizer.eos_token_id:
        break


 
 Autom obile Indust ry Ch anges : 
 
 The autom obile industry has under g one trem end ous changes since the in ception of the autom ob iles . The in vention of the electric motor , steam engine , and internal comb ust ion engine changed the way autom ob iles were power ed . The first autom ob iles were power ed by steam engines , but the development of these engines led to the creation of internal comb ust ion engines , which proved to be more efficient and reliable . 
 
 The First Autom ob iles : 
 
 The first autom ob iles were power ed by steam engines , but the development of these engines led to the creation of internal comb ust ion engines , which proved to be more efficient and reliable . The first autom ob iles were power ed by steam engines , but the development of these engines led to the creation of internal comb ust ion engines , which proved to be more efficient and reliable . 
 
 B irth of the Autom obile : 
 
 Fran k L . Scott is cred ited with the creation of the a

In [19]:
prompt = "Mass Production and the Model T"
inputs = tokenizer(prompt, return_tensors="pt")

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: val.to(device) for key, val in inputs.items()}
model.to(device)

# Generate tokens step-by-step
max_length = 500
generated = inputs["input_ids"]

for _ in range(max_length):
    outputs = model(generated)
    next_token_logits = outputs.logits[:, -1, :]
    next_token = torch.argmax(next_token_logits, dim=-1)
    generated = torch.cat((generated, next_token.unsqueeze(-1)), dim=-1)

    # Decode and print the latest token
    decoded_token = tokenizer.decode(next_token)
    print(decoded_token, end=" ", flush=True)

    # If the model generates the end of sentence token, stop early
    if next_token.item() == tokenizer.eos_token_id:
        break

: 
 
 A significant mil estone in the autom obile industry was the production and introduction of mass production techniques by Henry Ford in the early  2 0 th century . Ford ' s Model T , launched in  1 9 0 8 , became the first afford able car for the average American . The assembly line production method not only reduced the cost of manufact uring but also significantly increased the speed of production . By  1 9 2 7 , Ford had produced over  1 5 million Model T cars , making it the best - s elling car of its time . 
 
 Te chn ological Ad van c ements and the R ise of the Autom obile Indust ry : 
 
 T hr ough out the latter half of the  2 0 th century , techn ological adv anc ements led to significant changes in the autom obile industry . The introduction of gas oline - elect ric vehicles , automatic transm issions , and advanced safety features led to a more focused industry on providing customers with the best possible experience . The rise of car r ental services and mobile applic